<a href="https://colab.research.google.com/github/Shuaib11-Github/AI_Agents/blob/main/Medical_Image_Analysis_in_Healthcare_Using_AI_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Image Analysis in Healthcare Using AI Tools

## Introduction

Medical image analysis plays a vital role in healthcare for diagnosing conditions, monitoring treatments, and improving patient outcomes. Imagine uploading a medical image—like an X-ray, MRI scan, or ultrasound—and receiving detailed diagnostic insights in seconds.

This tutorial will guide you in building an **AI-powered medical image analysis application** capable of identifying conditions and providing diagnostic summaries using advanced AI tools like OpenAI's GPT-4, LangChain, Gradio, ChromaDB, and OpenAI Whisper.

By the end of this tutorial, you'll have a robust system that accepts a medical image as input and returns diagnostic information, optional patient care suggestions, and treatment recommendations.

---

## Prerequisites

Before we start, ensure you have the following libraries installed. If you are using Google Colab, run the following command:



In [ ]:
# Install required libraries
!pip install gradio langchain chromadb python-dotenv Pillow requests langchain_community tiktoken openai-whisper
!pip install openai==0.28  httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


## Step 1: Import Libraries and Set Up API Keys

We'll begin by importing necessary libraries and setting up the OpenAI API key.

In [ ]:
import os
import openai
import gradio as gr
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from PIL import Image
import base64
from datetime import datetime
import hashlib
import uuid
import chromadb
import whisper

# Set up OpenAI API key
from google.colab import userdata
OPENAI_API_KEY = userdata.get('openai')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY

## Step 2: Medical Image Analysis Class

We’ll create a `MedicalImageAnalyzer` class to process medical images and provide diagnostic summaries.

In [ ]:
class MedicalImageAnalyzer:
    def __init__(self):
        self.llm = OpenAI(temperature=0.7)
        self.embeddings = OpenAIEmbeddings()
        self.db = self._initialize_database()
        self.model = whisper.load_model("base")  # Load OpenAI Whisper for speech-to-text if needed.

    def _initialize_database(self):
        """Initialize ChromaDB for storing diagnostic information."""
        client = chromadb.Client()
        collection = client.get_or_create_collection(name="medical_diagnostics")
        return collection

    def analyze_image(self, image_path):
        """Analyze medical image using GPT-4 Vision."""
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode()

        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a professional radiologist and medical expert."},
                {"role": "user", "content": [
                    {"type": "text", "text": "Analyze the medical image below and provide diagnostic details."},
                    {"type": "image_url", "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_string}"
                    }}
                ]}
            ],
            max_tokens=500
        )
        return response.choices[0].message["content"]

    def generate_diagnosis(self, findings):
        """Generate detailed diagnosis based on image analysis."""
        template = """
        Based on the following findings, generate a detailed diagnosis and suggest next steps:
        {findings}

        Include:
        1. Diagnostic Summary
        2. Recommended Tests
        3. Possible Treatments
        4. Preventative Measures
        """
        prompt = PromptTemplate(input_variables=["findings"], template=template)
        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run(findings=findings)

## Step 3: Advanced Features

Extend `MedicalImageAnalyzer` to include treatment recommendations, follow-up plans, and patient care tips.

In [ ]:
class AdvancedMedicalImageAnalyzer(MedicalImageAnalyzer):
    def __init__(self):
        super().__init__()
        self.setup_templates()

    def setup_templates(self):
        """Setup templates for additional features."""
        self.templates = {
            'treatment': "Provide treatment options for the condition: {condition}.",
            'follow_up': "Suggest follow-up care and monitoring strategies for {condition}.",
            'care_tips': "Share patient care tips for managing {condition} at home.",
        }

    def get_treatment_options(self, condition):
        """Generate treatment options."""
        prompt = PromptTemplate(input_variables=["condition"], template=self.templates['treatment'])
        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run(condition=condition)

    def get_follow_up_plan(self, condition):
        """Get follow-up care plan."""
        prompt = PromptTemplate(input_variables=["condition"], template=self.templates['follow_up'])
        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run(condition=condition)

## Step 4: Gradio Interface

Create an interactive user interface using Gradio.

In [ ]:
def create_medical_interface():
    """Create a Gradio interface for medical image analysis."""
    analyzer = AdvancedMedicalImageAnalyzer()

    def process_image(image, include_treatment=False, include_follow_up=False):
        temp_path = "temp_image.jpg"
        image.save(temp_path)

        findings = analyzer.analyze_image(temp_path)
        result = f"Findings:\n{findings}\n\n"
        result += f"Diagnosis:\n{analyzer.generate_diagnosis(findings)}\n\n"

        if include_treatment:
            result += f"Treatment Options:\n{analyzer.get_treatment_options(findings)}\n\n"
        if include_follow_up:
            result += f"Follow-Up Plan:\n{analyzer.get_follow_up_plan(findings)}\n\n"

        return result

    interface = gr.Interface(
        fn=process_image,
        inputs=[
            gr.Image(type="pil", label="Upload Medical Image"),
            gr.Checkbox(label="Include Treatment Options"),
            gr.Checkbox(label="Include Follow-Up Plan"),
        ],
        outputs=gr.Textbox(label="Diagnostic Results", lines=20),
        title="AI Medical Image Analyzer",
        description="Upload a medical image for diagnostic insights, treatment suggestions, and care tips.",
        theme=gr.themes.Soft()
    )
    return interface

## Step 5: Main Application

Launch the application and share it.

In [ ]:
if __name__ == "__main__":
    interface = create_medical_interface()
    interface.launch(share=True, debug=True)

<ipython-input-3-f581fe689eee>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  self.llm = OpenAI(temperature=0.7)
<ipython-input-3-f581fe689eee>:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  self.embeddings = OpenAIEmbeddings()
100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 47.9MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d14abda4457a6679f1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary

This tutorial demonstrated how to create a **Medical Image Analysis application** using AI tools. By uploading a medical image, the app can analyze the image, generate diagnostic details, and suggest treatment and follow-up plans.

Feel free to enhance this application by integrating more medical databases, enabling voice-based inputs using Whisper, or providing multilingual support for global healthcare access.